# Complete the bad password generator

In the previous notebooks we've come close to completing the Bad Password generator shown below:

<img src="images/pass-gen-demo.gif" alt="animated demo of password generator" width="80%">

Let's finish it up to get some more practice with `traitlets` and linking a model to a widget.

## Model class

The code below contains most of the model. It adds two new traitlets and code to add special characteres and/or numbers to the password.

You should:

+ Make the model respond to changes in those new traitlets.
+ Test the model by setting the length and printing the generated password. 

**NOTE:** To keep the code a little shorter validation of the password length has been removed.

In [ ]:
# %load solutions/bad-password-final/bad-pass-model.py

import string
import random

import traitlets

SPECIAL_GROUPS = [',./;[', '!@#~%', '^&*()']


class PassGen(traitlets.HasTraits):
    """
    Class to represent state of the password generator and handle generation
    of password.
    """
    length = traitlets.Integer()
    password = traitlets.Unicode("password")
    
    # ↓↓↓↓↓↓↓↓ New traitlets ↓↓↓↓↓↓↓↓
    include_numbers = traitlets.Bool()
    special_character_groups = traitlets.Enum(SPECIAL_GROUPS,
                                              default_value=SPECIAL_GROUPS[0])

    def __init__(self):
        super(PassGen, self).__init__()
        pass

    @traitlets.observe('length')
    def generate_password(self, change):
        """
        Generate a password of the desired length including the user's chosen
        set of special characters and, if desired, including some numerals.
        """

        # Generate an initial password composed only of letters.
        new_pass = []
        for _ in range(self.length):
            new_pass.append(random.choice(string.ascii_letters))

        # Generate a list of indices for choosing which characters in the
        # initial password to replace, then shuffle it. We'll pop
        # elements off the list as we need them.
        order_for_replacements = list(range(self.length))
        random.shuffle(order_for_replacements)

        # Replace some of the letters with special characters
        n_special = random.randint(1, 3)
        for _ in range(n_special):
            loc = order_for_replacements.pop(0)
            new_pass[loc] = random.choice(self.special_character_groups)

        if self.include_numbers:
            # Number of digits to include.
            n_digits = random.randint(1, 3)
            for _ in range(n_digits):
                loc = order_for_replacements.pop(0)
                new_pass[loc] = random.choice(string.digits)

        self.password = ''.join(new_pass)

## View/widget

The code below is the `PassGenGUI` from the previous notebook. You need to do a few things to complete it:

+ Add the `better_toggles` (or at least the `Label` and the `toggles`) and the `numbers` widgets from the [layout exercises](07-container-exercises.ipynb). It is fine to add them to the `__init__` method of the class definition.
+ Link the values of those widgets to the appropriate traits of the model.

In [ ]:
# %load solutions/bad-password-final/bad-pass-gui.py

from ipywidgets import widgets

class PassGenGUI(widgets.VBox):
    def __init__(self):
        super(PassGenGUI, self).__init__()
        
        self._helpful_title = widgets.HTML('Generated password is:')
        self._password_text = widgets.HTML('', placeholder='No password generated yet')
        self._password_text.layout.margin = '0 0 0 20px'
        self._password_length = widgets.IntSlider(description='Length of password',
                                                  min=8, max=20,
                                                  style={'description_width': 'initial'})

        # Add additional GUI elements here
        
        children = [self._helpful_title, self._password_text, self._password_length, 
                    self._label, self._toggles, self._numbers]
        self.children = children
        
        self.model = PassGen()
        
        traitlets.link((self.model, 'password'), (self._password_text, 'value'))
        traitlets.link((self.model, 'length'), (self._password_length, 'value'))

        # You may need to do some more linking..
        
    @property
    def value(self):
        return self._password_text.value

Try out your generator by executing the cells below:

In [ ]:
pg = PassGenGUI()

In [ ]:
pg

## Distributing your bad password generator

Sending around notebooks with code doesn't work very well as a way to share your widgets with others. To see one way to distribute this code, take a look at the `password_generator_example` in the notebooks folder. It has two python files, [one for the model](password_generator_example/model.py) and [one for the GUI](password_generator_example/gui.py), and an [example notebook that imports them](password_generator_example/Widget example.ipynb).